

# Install necessary dependencies

In [1]:
!pip install -q accelerate -U
!pip install -q -U datasets
!pip install -q scipy
!pip install -q ipywidgets
!pip install -q wandb
!pip install -q transformers
!pip install -q torch
!pip install -q sentencepiece
!pip install -q tqdm
!pip install -q evaluate
!pip install -q rouge_score
!pip install --q tensorflow==2.14


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.8 

 # Import dependencies

In [2]:
# Data libraries
from datasets import load_dataset, Dataset, concatenate_datasets, list_metrics, load_metric

# preprocessing Libraries
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Model libraries
import transformers
import ast
import torch
import sklearn
from transformers import (
    T5Tokenizer, T5TokenizerFast, T5ForConditionalGeneration,
    AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM,
    DefaultDataCollator, T5ForQuestionAnswering, DataCollatorForSeq2Seq, pipeline, AutoModelForQuestionAnswering, BertTokenizer, BertForQuestionAnswering
)

# Metrics library
import evaluate
from evaluate import evaluator

# Visualization library
import wandb

# Warning libraries
import warnings
warnings.filterwarnings('ignore')

# Utility functions
import t5_utils as utils


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Data Visualization**

In [3]:
data_raw = load_dataset("imdb")
print(data_raw)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [6]:
data_raw['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [18]:
Input= data_raw['train'][11]['text']
print(f"text: \n{Input}\n---------------")

print(f"label:\n{data_raw['train'][11]['label']}")

text: 
I can't believe that those praising this movie herein aren't thinking of some other film. I was prepared for the possibility that this would be awful, but the script (or lack thereof) makes for a film that's also pointless. On the plus side, the general level of craft on the part of the actors and technical crew is quite competent, but when you've got a sow's ear to work with you can't make a silk purse. Ben G fans should stick with just about any other movie he's been in. Dorothy S fans should stick to Galaxina. Peter B fans should stick to Last Picture Show and Target. Fans of cheap laughs at the expense of those who seem to be asking for it should stick to Peter B's amazingly awful book, Killing of the Unicorn.
---------------
label:
0


# Global variables

In [72]:
DATASET = "imdb"
PREFIX = "sentiment analysis"
INPUT_COL = "text"
OUTPUT_COL = "label"
MODEL = "google/flan-t5-base"
TRAIN_SPLIT = "train"
VALIDATION_SPLIT = "test[:10000]"
TEST_SPLIT = "test[10000:20000]"



# Training args
MAX_STEPS = 1000
PER_DEVICE_TRAIN_BATCH_SIZE = 16
PER_DEVICE_EVAL_BATCH_SIZE = 16
SAVE_STEPS = 100
EVAL_STEPS = 100
LOGGING_STEPS = 100
CHECKPOINT_SAVED = 100
OPTIMIZER = "adamw_torch"
# adamw_hf, adamw_torch, adamw_torch_fused, adamw_apex_fused, adamw_anyprecision or adafactor.
LEARNING_RATE = 2.5e-5
COLLATOR = "seq2seq" # seq2seq or default

# for compute metrics
TASK = "text2text-generation"
METRICS = "rouge"


This two variables are defined and override from utility file to avoid error in training. (error : name "tokenizer" is not defined)

In [30]:
utils.metric = evaluate.load(METRICS)
utils.tokenizer = T5Tokenizer.from_pretrained(MODEL)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


This function takes data from HuggingFace or local data, split ( train, test, validation, train[:slicing] ), input_col, target_col, prefix(column or text)

In [31]:
train_data = utils.prepare_data(
                            DATASET,
                            TRAIN_SPLIT,
                            INPUT_COL,
                            OUTPUT_COL,
                            PREFIX)
print(train_data)

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 25000
})


In [14]:
eval_data = utils.prepare_data(
                           DATASET,
                           VALIDATION_SPLIT,
                           INPUT_COL,
                           OUTPUT_COL,
                           PREFIX)

print(eval_data)

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 100
})


In [73]:
test_data = utils.prepare_data(
                           DATASET,
                           TEST_SPLIT,
                           INPUT_COL,
                           OUTPUT_COL,
                           PREFIX)
print(test_data)

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 10000
})


Converting labels from integer to string

In [33]:
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)
eval_data = pd.DataFrame(eval_data)

train_data['target_text'] = train_data['target_text'].astype(str)
test_data['target_text'] = test_data['target_text'].astype(str)
eval_data['target_text'] = eval_data['target_text'].astype(str)

train_data = Dataset.from_pandas(train_data)
test_data = Dataset.from_pandas(test_data)
eval_data = Dataset.from_pandas(eval_data)


In [34]:
print(train_data['target_text'])

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',

In [35]:
max_source_length, max_target_length = utils.identify_max_lengths(
                                                                  train_data,
                                                                  eval_data,
                                                                  model = MODEL
                                                                  )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/25100 [00:00<?, ? examples/s]

Map:   0%|          | 0/25100 [00:00<?, ? examples/s]

In [36]:
print(f"Input max length : {max_source_length}")
print("--------------------------------")
print(f"Target max length : {max_target_length}")

Input max length : 512
--------------------------------
Target max length : 3


In [37]:
train_data_tokenized, eval_data_tokenized = utils.preprocessed_data(
                                                                    train_data,
                                                                    eval_data,
                                                                    max_source_length,
                                                                    max_target_length,
                                                                    model = MODEL
                                                                    )

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make s

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [38]:
eval_data_tokenized

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

# Training

Look at utility functions to get in-depth understanding of training function

In [39]:
utils.train_model(
           train_data_tokenized,
           eval_data_tokenized,
           project = "Finetunning-t5-Sentiment-Analysis2",
           model = MODEL,
           base_model_name = "T5",
           max_steps = MAX_STEPS,
           per_device_train_batch_size = PER_DEVICE_TRAIN_BATCH_SIZE,
           per_device_eval_batch_size = PER_DEVICE_EVAL_BATCH_SIZE,
           save_steps = SAVE_STEPS,
           eval_steps = EVAL_STEPS,
           logging_steps = LOGGING_STEPS,
           checkpoint_saved = CHECKPOINT_SAVED,
           optimizer = OPTIMIZER,
           learning_rate = LEARNING_RATE,
           collator = COLLATOR
           )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,0.811800,0.133922,86.000000,0.000000,86.000000,86.000000,8.000000
200,0.092500,0.113860,88.000000,0.000000,88.000000,88.000000,8.000000
300,0.081000,0.065290,92.000000,0.000000,92.000000,92.000000,8.000000
400,0.089000,0.087626,92.000000,0.000000,92.000000,92.000000,8.000000
500,0.077600,0.125144,86.000000,0.000000,86.000000,86.000000,8.000000
600,0.083800,0.106262,88.000000,0.000000,88.000000,88.000000,8.000000
700,0.081500,0.071669,91.000000,0.000000,91.000000,91.000000,8.000000
800,0.071000,0.077461,91.000000,0.000000,91.000000,91.000000,8.000000
900,0.079800,0.079110,91.000000,0.000000,91.000000,91.000000,8.000000
1000,0.085300,0.085285,91.000000,0.000000,91.000000,91.000000,8.000000


In [40]:
test_data[1]

{'input_text': 'sentiment analysis : In a nutshell the movie is about a gang war in the 1950\'s. Leon, the leader of the Deuces, starts the gang after his brother OD\'s on "junk". He vows to protect the neighborhood. The leader of the rival gang is just getting out of prison and wants revenge.<br /><br />The movie didn\'t really do it for me. The "Good Guys" weren\'t any more good than the "Bad Guys". Very little was shown to suggest that the Deuces really cared for the community. I suppose the writers were going for realism here, but I just didn\'t care which side won. None of the characters were likable, or even capable of drawing my sympathy.<br /><br />On the plus side the courtship between Annie and Bobby had some snappy dialog, and the acting overall was well done.',
 'target_text': '0'}

Evaluation on test data

In [ ]:
test_data[:5]

In [55]:
test_data

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 15000
})

In [74]:
task_evaluator = evaluator("text-classification")
metric = evaluate.load("accuracy", use_stemmer=True, use_aggregator=True)
results_for_finetuned = task_evaluator.compute(
model_or_pipeline="/content/drive/MyDrive/T5-Finetunning-t5-Sentiment-Analysis2/checkpoint-1000", data=test_data, metric=metric,
input_column="input_text", label_column="target_text",label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0}
                       )


Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/T5-Finetunning-t5-Sentiment-Analysis2/checkpoint-1000 and are newly initialized: ['classification_head.dense.weight', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'classification_head.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [75]:
print("Results of fine-tuned model on test data:\n", results_for_finetuned)

Results of fine-tuned model on test data:
 {'accuracy': 0.3381, 'total_time_in_seconds': 403.3417519300001, 'samples_per_second': 24.792870939221533, 'latency_in_seconds': 0.04033417519300001}


In [76]:
results_for_finetuned

{'accuracy': 0.3381,
 'total_time_in_seconds': 403.3417519300001,
 'samples_per_second': 24.792870939221533,
 'latency_in_seconds': 0.04033417519300001}

# **Evaluation**

# **Inference from Finetuned Flant5-base model**

In [78]:
input= test_data[1]['input_text']
print(f"text: \n{input}\n---------------")

# summarize dialogue
result =utils.inference('sentiment analysis ',
                input,
                fined_tuned_path="/content/drive/MyDrive/T5-Finetunning-t5-Sentiment-Analysis2/checkpoint-1000")


print(f"sentiment analysis:\n{result}")

text: 
sentiment analysis : In a nutshell the movie is about a gang war in the 1950's. Leon, the leader of the Deuces, starts the gang after his brother OD's on "junk". He vows to protect the neighborhood. The leader of the rival gang is just getting out of prison and wants revenge.<br /><br />The movie didn't really do it for me. The "Good Guys" weren't any more good than the "Bad Guys". Very little was shown to suggest that the Deuces really cared for the community. I suppose the writers were going for realism here, but I just didn't care which side won. None of the characters were likable, or even capable of drawing my sympathy.<br /><br />On the plus side the courtship between Annie and Bobby had some snappy dialog, and the acting overall was well done.
---------------


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


sentiment analysis:
0
